# // TO DO LIST//

use website: https://api.meteostat.net/#history with api key=ipeqdKHL

In [77]:
import pickle
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import requests
import lxml
import json
import os
import re
import numpy as np
from datetime import timedelta
api_key = 'ipeqdKHL'

In [78]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [79]:
dirName = 'HistoricalWeatherData'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print('Directory ', dirName, ' Created')
else:
    print('Directory ', dirName, ' already exists')

Directory  HistoricalWeatherData  already exists


In [80]:
def get_stadium_cities_wiki():
    url = 'https://en.wikipedia.org/wiki/List_of_current_National_Football_League_stadiums'
    table = pd.read_html(url)
    df = table[1]
    
    keep_cols = ['Name', 'Capacity', 'Location', 'Surface', 'Roof type', 'Team(s)', 'Opened']
    
    df = df[keep_cols]
    
    df['Capacity'] = df['Capacity'].astype(int)
    df['Surface'] = df['Surface'].replace(to_replace=r'\[\d{1,2}\]', value='', regex=True)
    df['Opened'] = df['Opened'].replace(to_replace=r'\[.*?\]', value='', regex=True)
    
    # need to fix this regex to properly divide the column into 2
#     df['Team(s)'] = df['Team(s)'].str.split(r'(.*[A-Za-z]+)([A-Z].*)', expand=True)
        
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_from_wiki.pkl')

In [81]:
def get_stadium_cities_nonwiki():
    url = 'https://geojango.com/pages/list-of-nfl-teams'
    table = pd.read_html(url)
    
    df = table[0]
    df[['City', 'State']] = df['Arena Location'].str.extract('^(?P<City>.+?),\s(?P<State>.+)', expand=True)
    team_abbrevs = {
            'Arizona Cardinals':'ARI',
            'Atlanta Falcons':'ATL',
            'Baltimore Ravens':'BAL',
            'Buffalo Bills':'BUF',
            'Carolina Panthers':'CAR',
            'Chicago Bears':'CHI',
            'Cincinnati Bengals':'CIN',
            'Cleveland Browns':'CLE',
            'Dallas Cowboys':'DAL',
            'Denver Broncos':'DEN',
            'Detroit Lions':'DET',
            'Green Bay Packers':'GB',
            'Houston Texans':'HOU',
            'Indianapolis Colts':'IND',
            'Jacksonville Jaguars':'JAX',
            'Kansas City Chiefs':'KC',
            'Los Angeles Rams':'LA',
            'Los Angeles Chargers':'LAC',
            'Miami Dolphins':'MIA',
            'Minnesota Vikings':'MIN',
            'New England Patriots':'NE',
            'New Orleans Saints':'NO',
            'New York Giants':'NYG',
            'New York Jets':'NYJ',
            'Oakland Raiders':'OAK',
            'Philadelphia Eagles':'PHI',
            'Pittsburgh Steelers':'PIT',
            'Seattle Seahawks':'SEA',
            'San Francisco 49ers':'SF',
            'Tampa Bay Buccaneers':'TB',
            'Tennessee Titans':'TEN',
            'Washington Redskins':'WAS'
            }
    
    df['Team Abbreviation'] = df['Team Name'].map(team_abbrevs)
    
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_not_from_wiki.pkl')
    
    
def get_state_abbrevs():
    url2 = 'https://www.softschools.com/social_studies/state_abbreviations/'
    table2 = pd.read_html(url2)
    df2 = table2[2]
    df2['State'] = df2['State'].str.title()
    df2.set_index('State', inplace=True)
    pd.to_pickle(df2, './state_abbreviations.pkl')
        

In [5]:
get_stadium_cities_nonwiki()
get_state_abbrevs()

In [82]:
stadiums_df = pd.read_pickle('./stadium_dataframe_not_from_wiki.pkl')
stadiums_df

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,City,State,Team Abbreviation
0,Arizona Cardinals,State Farm Stadium,"Glendale, Arizona",63400,2006,Glendale,Arizona,ARI
1,Atlanta Falcons,Mercedes-Benz Stadium,"Atlanta, Georgia",71000,2017,Atlanta,Georgia,ATL
2,Baltimore Ravens,M&T Bank Stadium,"Baltimore, Maryland",71008,1998,Baltimore,Maryland,BAL
3,Buffalo Bills,New Era Field,"Orchard Park, New York",71608,1973,Orchard Park,New York,BUF
4,Carolina Panthers,Bank of America Stadium,"Charlotte, North Carolina",75523,1996,Charlotte,North Carolina,CAR
5,Chicago Bears,Soldier Field,"Chicago, Illinois",61500,1924,Chicago,Illinois,CHI
6,Cincinnati Bengals,Paul Brown Stadium,"Cincinnati, Ohio",65515,2000,Cincinnati,Ohio,CIN
7,Cleveland Browns,First Energy Stadium,"Cleveland, Ohio",67895,1999,Cleveland,Ohio,CLE
8,Dallas Cowboys,At&T Stadium,"Arlington, Texas",80000,2009,Arlington,Texas,DAL
9,Denver Broncos,Empower Field at Mile High,"Denver, Colorado",76125,2001,Denver,Colorado,DEN


In [83]:
def get_station_id(city, key):
    url = 'https://api.meteostat.net/v1/stations/search?q={}&key={}'.format(city, key)
    response = requests.get(url)
    return response.json()

In [29]:
# test historical weather API
city = 'Baltimore'
station = get_station_id(city, api_key)['data'][0]['id']
station

'72406'

In [84]:
for row in stadiums_df['City']:
    print(row, get_station_id(row, api_key))
    print('\n\n')

Glendale {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72288', 'country': 'US', 'name': 'Burbank-Glendale-Pasadena'}, {'id': 'KGEU0', 'country': 'US', 'name': 'Glendale / Country Greens At Villa De Paz'}, {'id': 'KHVS0', 'country': 'US', 'name': 'Hartsville / Glendale Acres'}, {'id': 'KLUF0', 'country': 'US', 'name': 'Phoenix / Glendale / Webb'}, {'id': 'KORB0', 'country': 'US', 'name': 'Orr / Glendale'}]}



Atlanta {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72219', 'country': 'US', 'name': 'Hartsfield-Jackson Atlanta'}, {'id': 'KCVC0', 'country': 'US', 'name': 'Atlanta'}, {'id': 'KFTY0', 'country': 'US', 'name': 'Atlanta / Carroll Heights'}, {'id': 'KPDK0', 'country': 'US', 'name': 'Atlanta / Northwoods'}]}



Baltimore {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72406', 'country':

Oakland {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72493', 'country': 'US', 'name': 'Metro Oakland International  Airport'}, {'id': 'KWVL0', 'country': 'US', 'name': 'Waterville / Oakland'}, {'id': 'KRJD0', 'country': 'US', 'name': 'Ridgely / Oakland Acres'}]}



Philadelphia {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72408', 'country': 'US', 'name': 'Philadelphia International Airport'}, {'id': 'KPHD0', 'country': 'US', 'name': 'New Philadelphia / Schoenbrunn Estates'}, {'id': 'KPNE0', 'country': 'US', 'name': 'Philadelphia / Ashton Wooden Bridge'}]}



Pittsburgh {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72520', 'country': 'US', 'name': 'Greater Pittsburgh International'}]}



Santa Clara {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdie

In [85]:
def get_historical_data(dirName, city, station_id, key, timezone, start, end):
    url = 'https://api.meteostat.net/v1/history/hourly?station={}&start={}&end={}&time_zone={}&time_format=Y-m-d%20H:i&key={}'
    response = requests.get(url.format(station_id,start,end,timezone,key))
    weather_data = response.json()
    
    with open('{}\{}.json'.format(dirName, city), 'w') as f:
        json.dump(weather_data, f)
    
    print('Successfully wrote to json for ', city, ' from', start, ' to ', end)

In [32]:
get_historical_data(dirName, 'Glendale', 'KLUF0', api_key, 'Europe/London', '2019-01-01', '2019-01-05')

Successfully wrote to json for  Glendale  from 2019-01-01  to  2019-01-05


In [86]:
# need dictionary for cities and their correct station number
team_mapping = {
'Arizona Cardinals':'ARI',
'Atlanta Falcons':'ATL',
'Baltimore Ravens':'BAL',
'Buffalo Bills':'BUF',
'Carolina Panthers':'CAR',
'Chicago Bears':'CHI',
'Cincinnati Bengals':'CIN',
'Cleveland Browns':'CLE',
'Dallas Cowboys':'DAL',
'Denver Broncos':'DEN',
'Detroit Lions':'DET',
'Green Bay Packers':'GB',
'Houston Texans':'HOU',
'Indianapolis Colts':'IND',
'Jacksonville Jaguars':'JAX',
'Kansas City Chiefs':'KC',
'Los Angeles Rams':'LA',
'Los Angeles Chargers':'LAC',
'Miami Dolphins':'MIA',
'Minnesota Vikings':'MIN',
'New England Patriots':'NE',
'New Orleans Saints':'NO',
'New York Giants':'NYG',
'New York Jets':'NYJ',
'Oakland Raiders':'OAK',
'Philadelphia Eagles':'PHI',
'Pittsburgh Steelers':'PIT',
'Seattle Seahawks':'SEA',
'San Francisco 49ers':'SF',
'Tampa Bay Buccaneers':'TB',
'Tennessee Titans':'TEN',
'Washington Redskins':'WAS'
}

team_list = [v for k,v in team_mapping.items()]

In [139]:
cities = {'Glendale':['Phoenix',0,'ARI'], 'Atlanta':['Atlanta',0,'ATL'], 'Baltimore':['Baltimore',1,'BAL'],'Orchard Park':['Buffalo', 0,'BUF'],
          'Charlotte':['Charlotte',0,'CAR'], 'Chicago':['Chicago',0,'CHI'],'Cincinnati':['Cincinnati',0,'CIN'], 'Cleveland':['Cleveland',0,'CLE'],
          'Arlington':['Arlington',1,'DAL'], 'Denver':['Denver',1,'DEN'], 'Detroit':['Detroit',0,'DET'], 'Green Bay':['Oshkosh',0, 'GB'],
          'Houston':['Houston',0,'HOU'], 'Indianapolis':['Indianapolis',0,'IND'], 'Jacksonville':['Jacksonville',0,'JAX'], 'Kansas City':['Kansas City',0,'KC'],
          'Carson':['Hawthorne',0, 'LAC'], 'Los Angeles':['Los Angeles',0,'LA'], 'Miami Gardens':['Miami',0,'MIA'],
          'Minneapolis':['Minneapolis',0,'MIN'], 'Foxborough':['Boston',0,'NE'], 'New Orleans':['New Orleans',0,'NO'],
          'East Rutherford':['Newark', 0,'NYG_NYJ'], 'Oakland':['Oakland',0,'OAK'], 'Philadelphia':['Philadelphia',0,'PHI'],
          'Pittsburgh':['Pittsburgh',0,'PIT'], 'Santa Clara':['San Jose',1,'SF'], 'Seattle':['Seattle',0,'SEA'], 'Tampa':['Tampa',0,'TB'],
          'Nashville':['Nashville',0,'TEN'], 'Landover':['Washington',0,'WAS']}

for k,v in cities.items():
    print(k,v)

Glendale ['Phoenix', 0, 'ARI']
Atlanta ['Atlanta', 0, 'ATL']
Baltimore ['Baltimore', 1, 'BAL']
Orchard Park ['Buffalo', 0, 'BUF']
Charlotte ['Charlotte', 0, 'CAR']
Chicago ['Chicago', 0, 'CHI']
Cincinnati ['Cincinnati', 0, 'CIN']
Cleveland ['Cleveland', 0, 'CLE']
Arlington ['Arlington', 1, 'DAL']
Denver ['Denver', 1, 'DEN']
Detroit ['Detroit', 0, 'DET']
Green Bay ['Oshkosh', 0, 'GB']
Houston ['Houston', 0, 'HOU']
Indianapolis ['Indianapolis', 0, 'IND']
Jacksonville ['Jacksonville', 0, 'JAX']
Kansas City ['Kansas City', 0, 'KC']
Carson ['Hawthorne', 0, 'LAC']
Los Angeles ['Los Angeles', 0, 'LA']
Miami Gardens ['Miami', 0, 'MIA']
Minneapolis ['Minneapolis', 0, 'MIN']
Foxborough ['Boston', 0, 'NE']
New Orleans ['New Orleans', 0, 'NO']
East Rutherford ['Newark', 0, 'NYG_NYJ']
Oakland ['Oakland', 0, 'OAK']
Philadelphia ['Philadelphia', 0, 'PHI']
Pittsburgh ['Pittsburgh', 0, 'PIT']
Santa Clara ['San Jose', 1, 'SF']
Seattle ['Seattle', 0, 'SEA']
Tampa ['Tampa', 0, 'TB']
Nashville ['Nashville'

In [140]:
stadium_dict = {
    'ARI':['Bermuda grass', 'Retractable'],'ATL':['FieldTurf Revolution 360', 'Retractable'],'BAL':['Bermuda grass', 'Open'],
    'BUF':['A-Turf Titan 50', 'Open'],'CAR':['Bermuda grass', 'Open'],'CHI':['Kentucky bluegrass', 'Open'],
    'CIN':['UBU Speed Series S5-M Synthetic Turf', 'Open'],'CLE':['Kentucky bluegrass', 'Open'],'DAL':['Hellas Matrix Turf', 'Retractable'],
    'DEN':['Kentucky bluegrass', 'Open'],'DET':['FieldTurf Classic HD', 'Fixed'],'GB':['Desso GrassMaster', 'Open'],
    'HOU':['Hellas Matrix Turf', 'Retractable'],'IND':['Shaw Sports Momentum Pro', 'Retractable'],'JAX':['Bermuda grass', 'Open'],
    'KC':['Bermuda grass', 'Open'],'LAC':['Grass', 'Open'],'LA':['Grass', 'Open'],'MIA':['Platinum TE Paspalum', 'Open'],
    'MIN':['UBU Speed Series S5-M Synthetic Turf', 'Fixed'],'NE':['FieldTurf CORE','Open'],'NO':['FieldTurf Revolution 360', 'Fixed'],
    'NYG':['UBU Sports Speed Series S5-M Synthetic Turf', 'Open'],'NYJ':['UBU Sports Speed Series S5-M Synthetic Turf', 'Open'],
    'OAK':['Grass', 'Open'],'PHI':['Desso GrassMaster', 'Open'],'PIT':['Kentucky bluegrass', 'Open'],
    'SF':['Bermuda grass / Perennial Ryegrass mixture', 'Open'],'SEA':['FieldTurf Revolution 360', 'Open'],
    'TB':['Bermuda grass', 'Open'],'TEN':['Bermuda grass', 'Open'],'WAS':['Bermuda grass', 'Open']}

for k,v in stadium_dict.items():
    print(k,v)
    
with open('stadium_dict.pkl', 'wb') as handle:
    pickle.dump(stadium_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

ARI ['Bermuda grass', 'Retractable']
ATL ['FieldTurf Revolution 360', 'Retractable']
BAL ['Bermuda grass', 'Open']
BUF ['A-Turf Titan 50', 'Open']
CAR ['Bermuda grass', 'Open']
CHI ['Kentucky bluegrass', 'Open']
CIN ['UBU Speed Series S5-M Synthetic Turf', 'Open']
CLE ['Kentucky bluegrass', 'Open']
DAL ['Hellas Matrix Turf', 'Retractable']
DEN ['Kentucky bluegrass', 'Open']
DET ['FieldTurf Classic HD', 'Fixed']
GB ['Desso GrassMaster', 'Open']
HOU ['Hellas Matrix Turf', 'Retractable']
IND ['Shaw Sports Momentum Pro', 'Retractable']
JAX ['Bermuda grass', 'Open']
KC ['Bermuda grass', 'Open']
LAC ['Grass', 'Open']
LA ['Grass', 'Open']
MIA ['Platinum TE Paspalum', 'Open']
MIN ['UBU Speed Series S5-M Synthetic Turf', 'Fixed']
NE ['FieldTurf CORE', 'Open']
NO ['FieldTurf Revolution 360', 'Fixed']
NYG ['UBU Sports Speed Series S5-M Synthetic Turf', 'Open']
NYJ ['UBU Sports Speed Series S5-M Synthetic Turf', 'Open']
OAK ['Grass', 'Open']
PHI ['Desso GrassMaster', 'Open']
PIT ['Kentucky bluegra

In [141]:
with open('stadium_dict.pkl', 'rb') as handle:
    b = pickle.load(handle)
print(b)

{'ARI': ['Bermuda grass', 'Retractable'], 'ATL': ['FieldTurf Revolution 360', 'Retractable'], 'BAL': ['Bermuda grass', 'Open'], 'BUF': ['A-Turf Titan 50', 'Open'], 'CAR': ['Bermuda grass', 'Open'], 'CHI': ['Kentucky bluegrass', 'Open'], 'CIN': ['UBU Speed Series S5-M Synthetic Turf', 'Open'], 'CLE': ['Kentucky bluegrass', 'Open'], 'DAL': ['Hellas Matrix Turf', 'Retractable'], 'DEN': ['Kentucky bluegrass', 'Open'], 'DET': ['FieldTurf Classic HD', 'Fixed'], 'GB': ['Desso GrassMaster', 'Open'], 'HOU': ['Hellas Matrix Turf', 'Retractable'], 'IND': ['Shaw Sports Momentum Pro', 'Retractable'], 'JAX': ['Bermuda grass', 'Open'], 'KC': ['Bermuda grass', 'Open'], 'LAC': ['Grass', 'Open'], 'LA': ['Grass', 'Open'], 'MIA': ['Platinum TE Paspalum', 'Open'], 'MIN': ['UBU Speed Series S5-M Synthetic Turf', 'Fixed'], 'NE': ['FieldTurf CORE', 'Open'], 'NO': ['FieldTurf Revolution 360', 'Fixed'], 'NYG': ['UBU Sports Speed Series S5-M Synthetic Turf', 'Open'], 'NYJ': ['UBU Sports Speed Series S5-M Synthet

In [142]:
def get_all_station_id(city_dict, key):
    list_of_stations = []
    for k, v in city_dict.items():
        url = 'https://api.meteostat.net/v1/stations/search?q={}&key={}'.format(v[0], key)
        response = requests.get(url)
        station_json = response.json()['data'][v[1]]
        list_of_stations.append({k: [station_json,v[2]]})
        
        print('Got station id for ', k)
    
    with open('stations.pkl', 'wb') as f:
        pickle.dump(list_of_stations, f)
        

In [143]:
get_all_station_id(cities, api_key)

Got station id for  Glendale
Got station id for  Atlanta
Got station id for  Baltimore
Got station id for  Orchard Park
Got station id for  Charlotte
Got station id for  Chicago
Got station id for  Cincinnati
Got station id for  Cleveland
Got station id for  Arlington
Got station id for  Denver
Got station id for  Detroit
Got station id for  Green Bay
Got station id for  Houston
Got station id for  Indianapolis
Got station id for  Jacksonville
Got station id for  Kansas City
Got station id for  Carson
Got station id for  Los Angeles
Got station id for  Miami Gardens
Got station id for  Minneapolis
Got station id for  Foxborough
Got station id for  New Orleans
Got station id for  East Rutherford
Got station id for  Oakland
Got station id for  Philadelphia
Got station id for  Pittsburgh
Got station id for  Santa Clara
Got station id for  Seattle
Got station id for  Tampa
Got station id for  Nashville
Got station id for  Landover


In [144]:
with open('stations.pkl', 'rb') as f:
    stations_list = pickle.load(f)

In [145]:
for place in stations_list:
    for k, v in place.items():
        print(k, v)

Glendale [{'id': '72278', 'country': 'US', 'name': 'Phoenix Sky Harbor Airport'}, 'ARI']
Atlanta [{'id': '72219', 'country': 'US', 'name': 'Hartsfield-Jackson Atlanta'}, 'ATL']
Baltimore [{'id': 'KDMH0', 'country': 'US', 'name': 'Baltimore'}, 'BAL']
Orchard Park [{'id': '72528', 'country': 'US', 'name': 'Buffalo Airport'}, 'BUF']
Charlotte [{'id': '72314', 'country': 'US', 'name': 'Charlotte/Douglas International'}, 'CAR']
Chicago [{'id': '72530', 'country': 'US', 'name': 'Chicago O’hare Airport'}, 'CHI']
Cincinnati [{'id': '72421', 'country': 'US', 'name': 'Cincinnati/Northern Ky International'}, 'CIN']
Cleveland [{'id': '72524', 'country': 'US', 'name': 'Cleveland-Hopkins International'}, 'CLE']
Arlington [{'id': 'KGKY0', 'country': 'US', 'name': 'Arlington / Southland Acres'}, 'DAL']
Denver [{'id': '72565', 'country': 'US', 'name': 'Denver International Airport'}, 'DEN']
Detroit [{'id': '72537', 'country': 'US', 'name': 'Detroit Metropolitan'}, 'DET']
Green Bay [{'id': 'KOSH0', 'cou

In [146]:
for place in stations_list:
    for k, v in place.items():
        print(k,v[1])
        get_historical_data(dirName, v[1] + ' 2015to2019', v[0]['id'], api_key, 'Europe/London', '2015-01-01', '2019-12-31')

Glendale ARI
Successfully wrote to json for  ARI 2015to2019  from 2015-01-01  to  2019-12-31
Atlanta ATL
Successfully wrote to json for  ATL 2015to2019  from 2015-01-01  to  2019-12-31
Baltimore BAL
Successfully wrote to json for  BAL 2015to2019  from 2015-01-01  to  2019-12-31
Orchard Park BUF
Successfully wrote to json for  BUF 2015to2019  from 2015-01-01  to  2019-12-31
Charlotte CAR
Successfully wrote to json for  CAR 2015to2019  from 2015-01-01  to  2019-12-31
Chicago CHI
Successfully wrote to json for  CHI 2015to2019  from 2015-01-01  to  2019-12-31
Cincinnati CIN
Successfully wrote to json for  CIN 2015to2019  from 2015-01-01  to  2019-12-31
Cleveland CLE
Successfully wrote to json for  CLE 2015to2019  from 2015-01-01  to  2019-12-31
Arlington DAL
Successfully wrote to json for  DAL 2015to2019  from 2015-01-01  to  2019-12-31
Denver DEN
Successfully wrote to json for  DEN 2015to2019  from 2015-01-01  to  2019-12-31
Detroit DET
Successfully wrote to json for  DET 2015to2019  from

In [147]:
# combine json files into singular csv file
weather_path = os.getcwd() + '\\historicalweatherdata'

files = os.listdir(weather_path)

for name in files:
    print(name)

.ipynb_checkpoints
ARI 2015to2019.json
ARI.csv
ATL 2015to2019.json
ATL.csv
BAL 2015to2019.json
BAL.csv
BUF 2015to2019.json
BUF.csv
CAR 2015to2019.json
CAR.csv
CHI 2015to2019.json
CHI.csv
CIN 2015to2019.json
CIN.csv
CLE 2015to2019.json
CLE.csv
DAL 2015to2019.json
DAL.csv
DEN 2015to2019.json
DEN.csv
DET 2015to2019.json
DET.csv
GB 2015to2019.json
GB.csv
HOU 2015to2019.json
HOU.csv
IND 2015to2019.json
IND.csv
JAX 2015to2019.json
JAX.csv
KC 2015to2019.json
KC.csv
LA 2015to2019.json
LA.csv
LAC 2015to2019.json
LAC.csv
MIA 2015to2019.json
MIA.csv
MIN 2015to2019.json
MIN.csv
NE 2015to2019.json
NE.csv
NO 2015to2019.json
NO.csv
NYG.csv
NYG_NYJ 2015to2019.json
NYG_NYJ.csv
NYJ.csv
OAK 2015to2019.json
OAK.csv
PHI 2015to2019.json
PHI.csv
PIT 2015to2019.json
PIT.csv
SEA 2015to2019.json
SEA.csv
SF 2015to2019.json
SF.csv
TB 2015to2019.json
TB.csv
TEN 2015to2019.json
TEN.csv
WAS 2015to2019.json
WAS.csv


In [148]:
city_dict = dict(zip(stadiums_df['Team Abbreviation'],stadiums_df.City))
city_dict.update({'NYG_NYJ':'East Rutherford'})

for k,v in city_dict.items():
    print(k,v)

ARI Glendale
ATL Atlanta
BAL Baltimore
BUF Orchard Park
CAR Charlotte
CHI Chicago
CIN Cincinnati
CLE Cleveland
DAL Arlington
DEN Denver
DET Detroit
GB Green Bay
HOU Houston
IND Indianapolis
JAX Jacksonville
KC Kansas City
LAC Carson
LA Los Angeles
MIA Miami Gardens
MIN Minneapolis
NE Foxborough
NO New Orleans
NYG East Rutherford
NYJ East Rutherford
OAK Oakland
PHI Philadelphia
PIT Pittsburgh
SF Santa Clara
SEA Seattle
TB Tampa
TEN Nashville
WAS Landover
NYG_NYJ East Rutherford


<h2>For Windows</h2>

In [206]:
# regex for matching 'Atlanta.json' or 'Atlanta 2.json' and then can split that to save CSV file r'([^\\]*$)'
renamed_cols = {'time':'Time (GMT)', 'temperature':'Temperature (°C)', 'dewpoint':'Dewpoint (°C)', 'humidity':'Humidity (%)', 'precipitation':'Precipitation (mm)', 
                'windspeed':'Wind Speed (km/h)', 'winddirection':'Wind Direction (deg)', 'pressure':'Air Pressure (hPa)', 'City':'City', 'Team Abbreviation':'Team Abbreviation'}
weather_path = os.getcwd() + '\\historicalweatherdata'

files = os.listdir(weather_path)

cities = {'Glendale':['Phoenix',0,'ARI'], 'Atlanta':['Atlanta',0,'ATL'], 'Baltimore':['Baltimore',1,'BAL'],'Orchard Park':['Buffalo', 0,'BUF'],
          'Charlotte':['Charlotte',0,'CAR'], 'Chicago':['Chicago',0,'CHI'],'Cincinnati':['Cincinnati',0,'CIN'], 'Cleveland':['Cleveland',0,'CLE'],
          'Arlington':['Arlington',1,'DAL'], 'Denver':['Denver',1,'DEN'], 'Detroit':['Detroit',0,'DET'], 'Green Bay':['Oshkosh',0, 'GB'],
          'Houston':['Houston',0,'HOU'], 'Indianapolis':['Indianapolis',0,'IND'], 'Jacksonville':['Jacksonville',0,'JAX'], 'Kansas City':['Kansas City',0,'KC'],
          'Carson':['Hawthorne',0, 'LAC'], 'Los Angeles':['Los Angeles',0,'LA'], 'Miami Gardens':['Miami',0,'MIA'],
          'Minneapolis':['Minneapolis',0,'MIN'], 'Foxborough':['Boston',0,'NE'], 'New Orleans':['New Orleans',0,'NO'],
          'East Rutherford':['Newark', 0,'NYG_NYJ'], 'Oakland':['Oakland',0,'OAK'], 'Philadelphia':['Philadelphia',0,'PHI'],
          'Pittsburgh':['Pittsburgh',0,'PIT'], 'Santa Clara':['San Jose',1,'SF'], 'Seattle':['Seattle',0,'SEA'], 'Tampa':['Tampa',0,'TB'],
          'Nashville':['Nashville',0,'TEN'], 'Landover':['Washington',0,'WAS']}

for name in files:
    if '.json' in name:
        print(name)
        df = json.load(open(weather_path + '\\' + name))
        df = pd.DataFrame(df["data"])
        rename = re.findall(r'(.*)\ .*', name)[0]
        df['Team Abbreviation'] = rename
        df['City'] = df['Team Abbreviation'].map(city_dict)
        
        df['time'] = pd.to_datetime(df['time'])
        df = df.rename(columns=renamed_cols)
        drop_cols = ['Unnamed: 0', 'precipitation_3', 'precipitation_6', 'snowdepth', 'peakgust', 'condition', 'Local Time']
        df = df.drop(columns = [item for item in df.columns.to_list() if item in drop_cols])
        
        try:
            df['Field'] = stadium_dict[rename][0]
            df['Roof'] = stadium_dict[rename][1]
        except KeyError:
            # for the NYG-NYJ fields, which will be handled in the next cell
            pass
        df['Time (EST)'] = df['Time (GMT)'].dt.round('H') - timedelta(hours=4)
        
        # fills in new elements for missing time slots (mostly for DET, but we can see how many missing we have for each city now)
        df_len = len(df)-1
        df['added_time'] = 0
        for i in range(1, df_len):
            diff = df.loc[i, 'Time (EST)'] - df.loc[i-1, 'Time (EST)']
            if diff > pd.to_timedelta('1 hour'):
                hours = int(diff.total_seconds()/3600)
                for a in range(1, hours):
                    new_time_est = pd.to_timedelta('{} hour'.format(a)) + df.loc[i-1, 'Time (EST)']
                    new_time_gmt = pd.to_timedelta('{} hour'.format(a)) + df.loc[i-1, 'Time (GMT)']
                    team_abbrev = df.loc[i, 'Team Abbreviation']
                    city = df.loc[i, 'City']
                    field = df.loc[i, 'Field']
                    roof = df.loc[i, 'Roof']
                    df = df.append(pd.Series([new_time_gmt, new_time_gmt, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, team_abbrev, city, field, roof, new_time_est, 1], index=new_df.columns), ignore_index=True)

        df = df.sort_values(by=['Time (EST)'])
        df = df.reset_index()
        df = df.drop(columns=['index'])
        # linear interpolating missing data
        df = df.interpolate(method='linear')
        
        print('Saved ', rename + '.csv')
        df.to_csv(weather_path + '\\' + rename + '.csv', header=True, index=False)
        

ARI 2015to2019.json
Saved  ARI.csv
ATL 2015to2019.json
Saved  ATL.csv
BAL 2015to2019.json
Saved  BAL.csv
BUF 2015to2019.json
Saved  BUF.csv
CAR 2015to2019.json
Saved  CAR.csv
CHI 2015to2019.json
Saved  CHI.csv
CIN 2015to2019.json
Saved  CIN.csv
CLE 2015to2019.json
Saved  CLE.csv
DAL 2015to2019.json
Saved  DAL.csv
DEN 2015to2019.json
Saved  DEN.csv
DET 2015to2019.json
Saved  DET.csv
GB 2015to2019.json
Saved  GB.csv
HOU 2015to2019.json
Saved  HOU.csv
IND 2015to2019.json
Saved  IND.csv
JAX 2015to2019.json
Saved  JAX.csv
KC 2015to2019.json
Saved  KC.csv
LA 2015to2019.json
Saved  LA.csv
LAC 2015to2019.json
Saved  LAC.csv
MIA 2015to2019.json
Saved  MIA.csv
MIN 2015to2019.json
Saved  MIN.csv
NE 2015to2019.json
Saved  NE.csv
NO 2015to2019.json
Saved  NO.csv
NYG_NYJ 2015to2019.json
Saved  NYG_NYJ.csv
OAK 2015to2019.json
Saved  OAK.csv
PHI 2015to2019.json
Saved  PHI.csv
PIT 2015to2019.json
Saved  PIT.csv
SEA 2015to2019.json
Saved  SEA.csv
SF 2015to2019.json
Saved  SF.csv
TB 2015to2019.json
Saved

In [207]:
# fixes nyj and nyg together since they play in the same stadium
df_nyj = pd.read_csv(weather_path + '\\' + 'NYG_NYJ.csv')
df_nyg = pd.read_csv(weather_path + '\\' + 'NYG_NYJ.csv')
df_nyj['Team Abbreviation'] = 'NYJ'
df_nyg['Team Abbreviation'] = 'NYG'
df_nyj['Field'] = stadium_dict['NYJ'][0]
df_nyj['Roof'] = stadium_dict['NYJ'][1]
df_nyg['Field'] = stadium_dict['NYG'][0]
df_nyg['Roof'] = stadium_dict['NYG'][1]
df_nyj.to_csv(weather_path + '\\' + 'NYJ' + '.csv', header=True, index=False)
df_nyg.to_csv(weather_path + '\\' + 'NYG' + '.csv', header=True, index=False)

In [208]:
df = pd.read_csv(weather_path + '\\' + 'NE.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43825 entries, 0 to 43824
Data columns (total 15 columns):
Time (GMT)              43825 non-null object
time_local              43825 non-null object
Temperature (°C)        43825 non-null float64
Dewpoint (°C)           43825 non-null float64
Humidity (%)            43825 non-null float64
Precipitation (mm)      43824 non-null float64
Wind Speed (km/h)       43825 non-null float64
Wind Direction (deg)    43825 non-null float64
Air Pressure (hPa)      43825 non-null float64
Team Abbreviation       43825 non-null object
City                    43825 non-null object
Field                   43825 non-null object
Roof                    43825 non-null object
Time (EST)              43825 non-null object
added_time              43825 non-null int64
dtypes: float64(7), int64(1), object(7)
memory usage: 5.0+ MB


,Time (GMT),time_local,Temperature (°C),Dewpoint (°C),Humidity (%),Precipitation (mm),Wind Speed (km/h),Wind Direction (deg),Air Pressure (hPa),Team Abbreviation,City,Field,Roof,Time (EST),added_time
0,2015-01-01 00:00:00,2015-01-01 00:00,-3.3,-15.9,37.0,NaN,13.0,290.0,1023.2,NE,Foxborough,FieldTurf CORE,Open,2014-12-31 20:00:00,0
1,2015-01-01 01:00:00,2015-01-01 01:00,-3.9,-16.2,38.0,0.0,7.6,310.0,1022.9,NE,Foxborough,FieldTurf CORE,Open,2014-12-31 21:00:00,0
2,2015-01-01 02:00:00,2015-01-01 02:00,-3.9,-16.2,38.0,0.0,9.4,310.0,1022.2,NE,Foxborough,FieldTurf CORE,Open,2014-12-31 22:00:00,0
3,2015-01-01 03:00:00,2015-01-01 03:00,-5.0,-14.3,48.0,0.0,11.2,220.0,1021.7,NE,Foxborough,FieldTurf CORE,Open,2014-12-31 23:00:00,0
4,2015-01-01 04:00:00,2015-01-01 04:00,-5.6,-13.2,55.0,0.0,11.2,220.0,1021.0,NE,Foxborough,FieldTurf CORE,Open,2015-01-01 00:00:00,0
5,2015-01-01 05:00:00,2015-01-01 05:00,-5.6,-13.9,52.0,0.0,13.0,240.0,1020.2,NE,Foxborough,FieldTurf CORE,Open,2015-01-01 01:00:00,0
6,2015-01-01 06:00:00,2015-01-01 06:00,-5.6,-14.4,50.0,0.0,16.6,220.0,1019.3,NE,Foxborough,FieldTurf CORE,Open,2015-01-01 02:00:00,0
7,2015-01-01 07:00:00,2015-01-01 07:00,-5.6,-14.4,50.0,0.0,16.6,230.0,1018.4,NE,Foxborough,FieldTurf CORE,Open,2015-01-01 03:00:00,0
8,2015-01-01 08:00:00,2015-01-01 08:00,-5.6,-14.4,50.0,0.0,18.4,230.0,1017.9,NE,Foxborough,FieldTurf CORE,Open,2015-01-01 04:00:00,0
9,2015-01-01 09:00:00,2015-01-01 09:00,-4.4,-14.3,46.0,0.0,20.5,240.0,1017.0,NE,Foxborough,FieldTurf CORE,Open,2015-01-01 05:00:00,0


<h2>For Mac</h2>

In [53]:
# regex for matching 'Atlanta.json' or 'Atlanta 2.json' and then can split that to save CSV file r'([^\\]*$)'
renamed_cols = {'time':'Time (GMT)', 'temperature':'Temperature (°C)', 'dewpoint':'Dewpoint (°C)', 'humidity':'Humidity (%)', 'precipitation':'Precipitation (mm)', 
                'windspeed':'Wind Speed (km/h)', 'winddirection':'Wind Direction (deg)', 'pressure':'Air Pressure (hPa)', 'City':'City', 'Team Abbreviation':'Team Abbreviation'}
weather_path = os.getcwd() + '/historicalweatherdata'

files = os.listdir(weather_path)

for name in files:
    if '.json' in name:
        print(name)
        df = json.load(open(weather_path + '/' + name))
        df = pd.DataFrame(df["data"])
        rename = re.findall(r'(.*)\ .*', name)[0]
        df['Team Abbreviation'] = rename
        df['City'] = df['Team Abbreviation'].map(city_dict)
        
        df['time'] = pd.to_datetime(df['time'])
        df = df.rename(columns=renamed_cols)
        drop_cols = ['Unnamed: 0', 'precipitation_3', 'precipitation_6', 'snowdepth', 'peakgust', 'condition', 'Local Time']
        df = df.drop(columns = [item for item in df.columns.to_list() if item in drop_cols])
        try:
            df['Field'] = stadium_dict[rename][0]
            df['Roof'] = stadium_dict[rename][1]
        except KeyError:
            # for the NYG-NYJ fields, which will be handled in the next cell
            pass
        df['Time (EST)'] = df['Time (GMT)'] - timedelta(hours=4)
        print('Saved ', rename + '.csv')
        
        df.to_csv(weather_path + '/' + rename + '.csv', header=True, index=False)
        

ARI 2015to2019.json
Saved  ARI.csv
ATL 2015to2019.json
Saved  ATL.csv
BAL 2015to2019.json
Saved  BAL.csv
BUF 2015to2019.json
Saved  BUF.csv
CAR 2015to2019.json
Saved  CAR.csv
CHI 2015to2019.json
Saved  CHI.csv
CIN 2015to2019.json
Saved  CIN.csv
CLE 2015to2019.json
Saved  CLE.csv
DAL 2015to2019.json
Saved  DAL.csv
DEN 2015to2019.json
Saved  DEN.csv
DET 2015to2019.json
Saved  DET.csv
GB 2015to2019.json
Saved  GB.csv
HOU 2015to2019.json
Saved  HOU.csv
IND 2015to2019.json
Saved  IND.csv
JAX 2015to2019.json
Saved  JAX.csv
KC 2015to2019.json
Saved  KC.csv
LA 2015to2019.json
Saved  LA.csv
LAC 2015to2019.json
Saved  LAC.csv
MIA 2015to2019.json
Saved  MIA.csv
MIN 2015to2019.json
Saved  MIN.csv
NE 2015to2019.json
Saved  NE.csv
NO 2015to2019.json
Saved  NO.csv
NYG_NYJ 2015to2019.json
Saved  NYG_NYJ.csv
OAK 2015to2019.json
Saved  OAK.csv
PHI 2015to2019.json
Saved  PHI.csv
PIT 2015to2019.json
Saved  PIT.csv
SEA 2015to2019.json
Saved  SEA.csv
SF 2015to2019.json
Saved  SF.csv
TB 2015to2019.json
Saved

In [ ]:
df_nyj = pd.read_csv(weather_path + '/' + 'NYG_NYJ.csv')
df_nyg = pd.read_csv(weather_path + '/' + 'NYG_NYJ.csv')
df_nyj['Team Abbreviation'] = 'NYJ'
df_nyg['Team Abbreviation'] = 'NYG'
df_nyj['Field'] = stadium_dict['NYJ'][0]
df_nyj['Roof'] = stadium_dict['NYJ'][1]
df_nyg['Field'] = stadium_dict['NYG'][0]
df_nyg['Roof'] = stadium_dict['NYG'][1]
df_nyj.to_csv(weather_path + '/' + 'NYJ' + '.csv', header=True, index=False)
df_nyg.to_csv(weather_path + '/' + 'NYG' + '.csv', header=True, index=False)

<h2> After File Creation, some testing </h2>

In [209]:
df = pd.read_csv(weather_path + '//' + 'DET.csv')
df['Time (GMT)'] = pd.to_datetime(df['Time (GMT)'])
df['Time (EST)'] = pd.to_datetime(df['Time (EST)'])
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43985 entries, 0 to 43984
Data columns (total 15 columns):
Time (GMT)              43985 non-null datetime64[ns]
time_local              43985 non-null object
Temperature (°C)        43985 non-null float64
Dewpoint (°C)           43985 non-null float64
Humidity (%)            43985 non-null float64
Precipitation (mm)      43984 non-null float64
Wind Speed (km/h)       43985 non-null float64
Wind Direction (deg)    43985 non-null float64
Air Pressure (hPa)      43985 non-null float64
Team Abbreviation       43985 non-null object
City                    43985 non-null object
Field                   43985 non-null object
Roof                    43985 non-null object
Time (EST)              43985 non-null datetime64[ns]
added_time              43985 non-null int64
dtypes: datetime64[ns](2), float64(7), int64(1), object(5)
memory usage: 5.0+ MB


,Time (GMT),time_local,Temperature (°C),Dewpoint (°C),Humidity (%),Precipitation (mm),Wind Speed (km/h),Wind Direction (deg),Air Pressure (hPa),Team Abbreviation,City,Field,Roof,Time (EST),added_time
0,2015-01-01 00:00:00,2015-01-01 00:00,-8.3,-17.9,46.0,NaN,22.3,220.0,1023.70,DET,Detroit,FieldTurf Classic HD,Fixed,2014-12-31 20:00:00,0
1,2015-01-01 01:00:00,2015-01-01 01:00,-8.3,-17.3,48.5,0.0,29.7,215.0,1022.75,DET,Detroit,FieldTurf Classic HD,Fixed,2014-12-31 21:00:00,0
2,2015-01-01 02:00:00,2015-01-01 02:00,-8.3,-16.7,51.0,0.0,37.1,210.0,1021.80,DET,Detroit,FieldTurf Classic HD,Fixed,2014-12-31 22:00:00,0
3,2015-01-01 03:00:00,2015-01-01 03:00,-8.3,-16.2,53.0,0.0,38.9,220.0,1021.30,DET,Detroit,FieldTurf Classic HD,Fixed,2014-12-31 23:00:00,0
4,2015-01-01 04:00:00,2015-01-01 04:00,-7.8,-15.5,54.0,0.0,29.5,220.0,1021.00,DET,Detroit,FieldTurf Classic HD,Fixed,2015-01-01 00:00:00,0
5,2015-01-01 05:00:00,2015-01-01 05:00,-7.8,-15.1,56.0,0.0,25.9,220.0,1020.20,DET,Detroit,FieldTurf Classic HD,Fixed,2015-01-01 01:00:00,0
6,2015-01-01 06:00:00,2015-01-01 06:00,-7.8,-14.4,59.0,0.0,27.7,220.0,1019.50,DET,Detroit,FieldTurf Classic HD,Fixed,2015-01-01 02:00:00,0
7,2015-01-01 07:00:00,2015-01-01 07:00,-7.8,-14.4,59.0,0.0,26.8,215.0,1019.25,DET,Detroit,FieldTurf Classic HD,Fixed,2015-01-01 03:00:00,0
8,2015-01-01 08:00:00,2015-01-01 08:00,-7.8,-14.4,59.0,0.0,25.9,210.0,1019.00,DET,Detroit,FieldTurf Classic HD,Fixed,2015-01-01 04:00:00,0
9,2015-01-01 09:00:00,2015-01-01 09:00,-7.8,-15.1,56.0,0.0,24.1,210.0,1018.70,DET,Detroit,FieldTurf Classic HD,Fixed,2015-01-01 05:00:00,0


In [210]:
df.dtypes

Time (GMT)              datetime64[ns]
time_local                      object
Temperature (°C)               float64
Dewpoint (°C)                  float64
Humidity (%)                   float64
Precipitation (mm)             float64
Wind Speed (km/h)              float64
Wind Direction (deg)           float64
Air Pressure (hPa)             float64
Team Abbreviation               object
City                            object
Field                           object
Roof                            object
Time (EST)              datetime64[ns]
added_time                       int64
dtype: object

In [211]:
df.loc[(df['Time (EST)'].dt.day==16) & (df['Time (EST)'].dt.month==12) & (df['Time (EST)'].dt.year==2017)]

,Time (GMT),time_local,Temperature (°C),Dewpoint (°C),Humidity (%),Precipitation (mm),Wind Speed (km/h),Wind Direction (deg),Air Pressure (hPa),Team Abbreviation,City,Field,Roof,Time (EST),added_time
26072,2017-12-16 03:53:00,2017-12-16 03:53,-2.000000,-6.900000,69.000000,0.0,19.000000,230.000000,1032.222309,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 00:00:00,0
26073,2017-12-16 04:53:00,2017-12-16 04:53,-2.000000,-6.000000,74.000000,0.0,13.000000,220.000000,1032.225561,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 01:00:00,0
26074,2017-12-16 05:53:00,2017-12-16 05:53:00,-1.500000,-5.450000,74.500000,0.0,19.500000,225.000000,1032.228812,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 02:00:00,1
26075,2017-12-16 06:43:00,2017-12-16 06:43,-1.000000,-4.900000,75.000000,0.0,26.000000,230.000000,1032.232063,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 03:00:00,0
26076,2017-12-16 07:39:00,2017-12-16 07:39,-1.000000,-4.000000,80.000000,0.0,26.000000,220.000000,1032.235314,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 04:00:00,0
26077,2017-12-16 08:17:00,2017-12-16 08:17,-1.000000,-4.000000,80.000000,0.0,24.000000,210.000000,1032.238565,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 04:00:00,0
26078,2017-12-16 09:17:00,2017-12-16 09:17:00,-0.333333,-3.966667,76.666667,0.0,23.333333,213.333333,1032.241816,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 05:00:00,1
26079,2017-12-16 10:17:00,2017-12-16 10:17:00,0.333333,-3.933333,73.333333,0.0,22.666667,216.666667,1032.245067,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 06:00:00,1
26080,2017-12-16 10:53:00,2017-12-16 10:53,1.000000,-3.900000,70.000000,0.0,22.000000,220.000000,1032.248318,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 07:00:00,0
26081,2017-12-16 11:53:00,2017-12-16 11:53,1.000000,-3.900000,70.000000,0.0,22.000000,220.000000,1032.251570,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 08:00:00,0


In [212]:
df.describe()

,Temperature (°C),Dewpoint (°C),Humidity (%),Precipitation (mm),Wind Speed (km/h),Wind Direction (deg),Air Pressure (hPa),added_time
count,43985.000000,43985.000000,43985.000000,43984.000000,43985.000000,43985.000000,43985.000000,43985.000000
mean,10.835869,4.701479,68.242344,0.102108,14.189078,193.899113,1017.793101,0.021757
std,11.213639,10.575959,16.609592,0.746247,8.617751,94.565847,7.694139,0.145892
min,-25.000000,-31.200000,12.000000,0.000000,0.000000,1.000000,988.600000,0.000000
25%,2.000000,-3.200000,57.000000,0.000000,7.600000,130.000000,1012.700000,0.000000
50%,11.100000,5.100000,69.000000,0.000000,13.000000,200.000000,1017.500000,0.000000
75%,20.600000,13.900000,81.000000,0.000000,19.000000,270.000000,1022.900000,0.000000
max,35.600000,25.700000,100.000000,52.800000,63.000000,360.000000,1046.200000,1.000000


In [198]:
new_df = df.loc[(df['Time (EST)'].dt.day==16) & (df['Time (EST)'].dt.month==12) & (df['Time (EST)'].dt.year==2017)]
new_df = new_df.reset_index()
new_df = new_df.drop(columns=['index'])
df_len = len(new_df)-1
new_df['added_time'] = 0
for i in range(1, df_len):
    diff = new_df.loc[i, 'Time (EST)'] - new_df.loc[i-1, 'Time (EST)']
    if diff > pd.to_timedelta('1 hour'):
        hours = int(diff.total_seconds()/3600)
        print(diff, new_df.loc[i, 'Time (EST)'], new_df.loc[i-1, 'Time (EST)'])
        for a in range(1, hours):
            new_time_est = pd.to_timedelta('{} hour'.format(a)) + new_df.loc[i-1, 'Time (EST)']
            new_time_gmt = pd.to_timedelta('4 hour') + new_df.loc[i-1, 'Time (EST)']
            team_abbrev = new_df.loc[i, 'Team Abbreviation']
            city = new_df.loc[i, 'City']
            field = new_df.loc[i, 'Field']
            roof = new_df.loc[i, 'Roof']
            new_df = new_df.append(pd.Series([new_time_gmt, new_time_gmt, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, team_abbrev, city, field, roof, new_time_est, 1], index=new_df.columns), ignore_index=True)
# dfObj.append(pd.Series(['Raju', 21, 'Bangalore', 'India'], index=dfObj.columns ), ignore_index=True)
new_df = new_df.sort_values(by=['Time (EST)'])
new_df = new_df.reset_index()
new_df = new_df.drop(columns=['index'])
# linear interpolating missing data
new_df = new_df.interpolate(method='linear')
        
new_df

0 days 02:00:00 2017-12-16 03:00:00 2017-12-16 01:00:00
0 days 03:00:00 2017-12-16 07:00:00 2017-12-16 04:00:00
0 days 02:00:00 2017-12-16 14:00:00 2017-12-16 12:00:00
0 days 03:00:00 2017-12-16 18:00:00 2017-12-16 15:00:00


,Time (GMT),time_local,Temperature (°C),Dewpoint (°C),Humidity (%),Precipitation (mm),Wind Speed (km/h),Wind Direction (deg),Air Pressure (hPa),Team Abbreviation,City,Field,Roof,Time (EST),added_time
0,2017-12-16 03:53:00,2017-12-16 03:53,-2.000000,-6.900000,69.000000,0.0,19.000000,230.000000,1032.320635,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 00:00:00,0
1,2017-12-16 04:53:00,2017-12-16 04:53,-2.000000,-6.000000,74.000000,0.0,13.000000,220.000000,1032.324275,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 01:00:00,0
2,2017-12-16 05:00:00,2017-12-16 05:00:00,-1.500000,-5.450000,74.500000,0.0,19.500000,225.000000,1032.326095,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 02:00:00,1
3,2017-12-16 06:43:00,2017-12-16 06:43,-1.000000,-4.900000,75.000000,0.0,26.000000,230.000000,1032.327915,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 03:00:00,0
4,2017-12-16 07:39:00,2017-12-16 07:39,-1.000000,-4.000000,80.000000,0.0,26.000000,220.000000,1032.331555,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 04:00:00,0
5,2017-12-16 08:17:00,2017-12-16 08:17,-1.000000,-4.000000,80.000000,0.0,24.000000,210.000000,1032.335195,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 04:00:00,0
6,2017-12-16 08:00:00,2017-12-16 08:00:00,-0.333333,-3.966667,76.666667,0.0,23.333333,213.333333,1032.336409,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 05:00:00,1
7,2017-12-16 08:00:00,2017-12-16 08:00:00,0.333333,-3.933333,73.333333,0.0,22.666667,216.666667,1032.337622,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 06:00:00,1
8,2017-12-16 10:53:00,2017-12-16 10:53,1.000000,-3.900000,70.000000,0.0,22.000000,220.000000,1032.338835,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 07:00:00,0
9,2017-12-16 11:53:00,2017-12-16 11:53,1.000000,-3.900000,70.000000,0.0,22.000000,220.000000,1032.342475,DET,Detroit,FieldTurf Classic HD,Fixed,2017-12-16 08:00:00,0
